# Train-test split
Author: Michal Stary


In this notebook we perform train-test split of our original datasets' files. 
Since we need to cope with multiple records of most of the compounds, such a split needs to be done carefuly. 

In [1]:
# path to datasets
mona_path = "/storage/brno1-cerit/projects/msml/data/MoNA-export-GC-MS_Spectra.msp"
nist_path = "/storage/brno6/home/xstary1/raims/data/20210925_NIST_EI_MS_cleaned.msp"

In [2]:
import numpy as np

from matchms.importing import load_from_msp
from matchms.exporting import save_as_msp as save_as_msp_orig

# REDEFINE save_as_msp function to avoid problems with nested comments
def save_as_msp(ds, path):
    for rec in ds:
        if "comments" in rec.metadata:
            rec.set("comment", rec.get("comments"))
            rec._metadata.pop("comments")
    save_as_msp_orig(ds, path)

In [3]:
mona = [*load_from_msp(mona_path)]
nist = [*load_from_msp(nist_path)]


In [4]:
data = dict()
data["mona"] = dict()
data["nist"] = dict()

In [5]:
# fix seed for now
np.random.seed(42)

## Naive solution - Random split
randomly assign into partitions

In [6]:
def random_split(size, split1=.8, split2=.9):
    split = int(size*split1)
    split2 = int(size*split2)

    ind = np.zeros(size)
    ind[split:split2] = 1
    ind[split2:] = 2

    
    np.random.shuffle(ind)

    train_ind = np.where(ind==0)[0]
    val_ind = np.where(ind==1)[0]
    test_ind = np.where(ind==2)[0]

    return train_ind, val_ind, test_ind

def get_sub(docs, indcs):
    return [docs[i] for i in indcs]

def do_random_split(mona, nist):
    data["mona"]["random_split_8_9"] = [get_sub(mona, indcs) for indcs in random_split(len(mona))]
    data["nist"]["random_split_8_9"] = [get_sub(nist, indcs) for indcs in random_split(len(nist))]

do_random_split(mona, nist)


### Notes
Unfortunately, the simple naive solution does not take in account the repetitive records of the same compound in dataset. 
As a result, the effect of such a split is that compounds present in test set can overlap with compounds in train set. This violates the standard splitting conditions. 

SCENARIO: I may or may not have the compound in DB, but it was measured imperfecly in experiment. 

# Advanced solutions
To simulate and assess performance without overlapping sets, we need to use records' attribute that represents what compound they represent.  

Records without this attribute are skipped for now.


In [7]:
# get mapping and first occurences of each unique compound
def get_map(ds, unkey="inchikey", parseuk_fn=lambda x: x):
    map_ = dict()
    dropped = 0
    for i, rec in enumerate(ds):
        if unkey not in rec.metadata or rec.metadata[unkey] == "nan":
            dropped +=1
            continue
        
        if parseuk_fn(rec.metadata[unkey]) not in map_:
            map_[parseuk_fn(rec.metadata[unkey])] = [rec]
        else:
            map_[parseuk_fn(rec.metadata[unkey])].append(rec)
            
    print(f"Dropped {dropped} out of {len(ds)} records due to missing unique key")
    return map_

print("**MONA***")
mona_map = get_map(mona, "inchikey", lambda x: x.split("-")[0])
print(f"Unique keys {len(mona_map)}")


print()
print("***NIST***")
nist_map = get_map(nist, "inchikey", lambda x: x.split("-")[0])
print(f"Unique keys {len(nist_map)}")


**MONA***
Dropped 6 out of 18898 records due to missing unique key
Unique keys 9143

***NIST***
Dropped 60163 out of 350618 records due to missing unique key
Unique keys 243413


## Put aside some compounds completely
put aside few compunds for the very ultimate evaluation of working with previously unseen compounds


We have 2 options how to do this:

1. remove all records of selected compounds from datasets 
2. use dedicated extra dataset and make sure that its compounds are not in the datasets

### First approach - select & remove

In [8]:
# random selection of compounds 
def select_random(ds_map, partition=.03):
    split = int(len(ds_map)*partition)
    ind = np.zeros(len(ds_map))
    ind[split:] = 1
    np.random.shuffle(ind)
    
    map_kept = dict()
    map_left = dict()
    for i, key in zip(ind, ds_map):
        if i != 0:
            map_kept[key] = ds_map[key]  
        else:
            map_left[key] = ds_map[key] 
            
    return map_kept, map_left

def do_select_random(mona_map, nist_map):
    mona_map, mona_map_left = select_random(mona_map)
    data["mona"]["left"] = [[item for sublist in mona_map_left.values() for item in sublist]]

    nist_map, nist_map_left = select_random(nist_map)
    data["nist"]["left"] = [[item for sublist in nist_map_left.values() for item in sublist]]

    return mona_map, nist_map

mona_map, nist_map = do_select_random(mona_map, nist_map)

### Second approach - obtain, check, (remove)
We will use small hi-res datasets from RECETOX

we can either remove mutual compounds from base datasets (MONA/NIST) or from these extra datasets. 

In [9]:
# path to extra datasets
rcx1_path = "/storage/brno1-cerit/projects/msml/data/recetox_gc-ei_ms_20201028.msp"
rcx2_path = "/storage/brno1-cerit/projects/msml/data/rcx_gc-orbitrap_metabolites_20210817.msp"


rcx1 = [*load_from_msp(rcx1_path)]
rcx2 = [*load_from_msp(rcx2_path)]

rcx1_map = get_map(rcx1)
rcx2_map = get_map(rcx2)



Dropped 0 out of 386 records due to missing unique key
Dropped 0 out of 265 records due to missing unique key


In [10]:
# substract from ds1 all records that have same InChiKey as any record in ds2
def substract_by_key(ds1_map, ds2_map):
    ds1_attr = set(ds1_map.keys())
    ds2_attr = set(ds2_map.keys())
    left_attr = ds1_attr.difference(ds2_attr)    
    #print(len(ds1_attr), len(left_attr))
    return left_attr
#rcx1[0].metadata["inchikey"]
def get_list_by_keys(keys, map_):
    res = []
    for key in keys:
        res += map_[key]
    return res

def do_small():
    data["mona"]["rcx1_no_mona"] = [get_list_by_keys(substract_by_key(rcx1_map, mona_map), rcx1_map)]
    data["mona"]["rcx2_no_mona"] = [get_list_by_keys(substract_by_key(rcx2_map, mona_map), rcx2_map)]
    
    data["nist"]["rcx1_no_nist"] = [get_list_by_keys(substract_by_key(rcx1_map, nist_map), rcx1_map)]
    data["nist"]["rcx2_no_nist"] = [get_list_by_keys(substract_by_key(rcx2_map, nist_map), rcx2_map)]
do_small()

# Scenario 2 - dealing solely with known compounds
Firstly, we divide our dataset into train-val-test partitions to simulate prediction of missing parts of compounds that are part of our database.

We do that by leveraging the duplicity of compounds in dababases:
        
    All first occurences of compounds in dataset are put into train set
    Left out records are then splitted by compounds and put into validation and test sets. 

In [11]:
def get_sub_nested(map_, indcs):
    keys = np.array(list(map_.keys()))[indcs]
    return [item for sublist in [map_[i] for i in keys] for item in sublist]

def get_first_others(ds_map):
    first = []
    others_map = dict()
    for key in ds_map:
        first.append(ds_map[key][0])
        if len(ds_map[key]) > 1:
            others_map[key] = ds_map[key][1:]
    return first, others_map

def do_first_others_split(mona_map, nist_map):
    mona_first, mona_others_map = get_first_others(mona_map)
    nist_first, nist_others_map = get_first_others(nist_map)
    
    # split by compounds
    mona_others_ind = random_split(len(mona_others_map.keys()), 0,.5)
    nist_others_ind = random_split(len(nist_others_map.keys()), 0,.5)
    
    data["mona"]["in_database"] = [mona_first] + [get_sub_nested(mona_others_map, indcs) for indcs in mona_others_ind][1:]
    data["nist"]["in_database"] = [nist_first] + [get_sub_nested(nist_others_map, indcs) for indcs in nist_others_ind][1:]

do_first_others_split(mona_map, nist_map)

# Scenario 3 - dealing solely with unknown compounds
Secodly, we divide our dataset into train-val-test partitions to simulate prediction of missing parts of compounds that are not in our database.

    Dataset is splitted so either only one record per compound is kept or that repetetive records of the same compound are only in one partition. 

## Keep unique only random split
keeps only first occurence of each compound in ds

randomly assign into partitions

In [12]:
split_mona = random_split(len(mona_map))
split_nist = random_split(len(nist_map))

In [13]:
def get_first_occ(ds_map):
    first = []
    for key in ds_map:
        first.append(ds_map[key][0])
    return first

def do_first_only_random_split(mona_map, nist_map):
    data["mona"]["unique_inchi"] = [get_sub(get_first_occ(mona_map), indcs) for indcs in split_mona]
    data["nist"]["unique_inchi"] = [get_sub(get_first_occ(nist_map), indcs) for indcs in split_nist]

do_first_only_random_split(mona_map, nist_map)

## Duplicates in set only split
assure that duplicate records of any compounds occurs only in one partition

SCENARIO: I DO NOT have the compound in DB, but it was measured imperfecly in experiment and.

In [14]:


def do_duplicates_in_partition_only_split(mona_map, nist_map):
    data["mona"]["de_novo"] = [get_sub_nested(mona_map, indcs) for indcs in split_mona]
    data["nist"]["de_novo"] = [get_sub_nested(nist_map, indcs) for indcs in split_nist]

do_duplicates_in_partition_only_split(mona_map, nist_map)

## Inspections

In [15]:
mona[1].metadata

{'name': '2,4-DINITROPHENOL',
 'synon': '$:00in-source',
 'db#': 'JP000002',
 'inchikey': 'UFBJCMHMOXMLKC-UHFFFAOYSA-N',
 'spectrum_type': 'MS1',
 'instrument_type': 'EI-B',
 'instrument': 'VARIAN MAT-44',
 'ion_mode': 'P',
 'formula': 'C6H4N2O5',
 'mw': '184',
 'exactmass': '184.01202122799998',
 'smiles': '[O-1][N+1](=O)c(c1)cc([N+1]([O-1])=O)c(O)c1',
 'inchi': 'InChI=1S/C6H4N2O5/c9-6-2-1-4(7(10)11)3-5(6)8(12)13/h1-3,9H',
 'computed smiles': 'C1=C(C=C(C(=C1)O)N(=O)=O)N(=O)=O',
 'accession': 'JP000002',
 'date': '2016.01.19 (Created 2008.10.21, modified 2011.05.06)',
 'author': 'KOGA M, UNIV. OF OCCUPATIONAL AND ENVIRONMENTAL HEALTH',
 'license': 'CC BY-NC-SA',
 'exact mass': '184.01202',
 'ionization energy': '70 eV',
 'ion type': '[M]+*',
 'splash': 'splash10-0w33-9300000000-b1719d21caddddc047cb',
 'submitter': 'University of Tokyo Team (Faculty of Engineering, University of Tokyo)',
 'mona rating': '3.75',
 'num peaks': '64'}

In [16]:
data["nist"].keys()

dict_keys(['random_split_8_9', 'left', 'rcx1_no_nist', 'rcx2_no_nist', 'in_database', 'unique_inchi', 'de_novo'])

In [17]:
[*map(len, data["nist"]["random_split_8_9"])]

[280494, 35062, 35062]

In [18]:
[*map(len,data["nist"]["de_novo"])]

[225611, 28108, 27980]

In [19]:
[*map(len,data["nist"]["in_database"])]

[236111, 22978, 22610]

In [20]:
from collections import Counter

print("(n, k) tuples, where n=number of duplicates and k=number of compounds with k duplicates")
Counter([len(nist_map[k]) for k in nist_map]).most_common()


(n, k) tuples, where n=number of duplicates and k=number of compounds with k duplicates


[(1, 210330),
 (2, 15066),
 (3, 5629),
 (4, 2847),
 (5, 1447),
 (6, 427),
 (7, 153),
 (9, 69),
 (8, 58),
 (10, 34),
 (11, 14),
 (12, 14),
 (13, 10),
 (14, 4),
 (19, 3),
 (15, 2),
 (17, 2),
 (24, 1),
 (16, 1)]

## Save into files

In [21]:
import os
def save(data, path):
    for ds in data:
        #os.makedirs(path + ds, exist_ok=True)
        for kind in data[ds]:
            os.makedirs(f"{path}/{ds}/{kind}", exist_ok=True)
            for part, recs in zip(["train", "val", "test"], data[ds][kind]):
                save_as_msp(recs, f"{path}/{ds}/{kind}/{part}.msp")

save(data, "/storage/brno6/home/xstary1/raims/data")
#save(data, "/storage/brno1-cerit/projects/msml/data/split")